In [35]:
## Metadata Extraction

import os
from PIL import Image, PngImagePlugin
import exifread
import pandas as pd


def extract_exif_data(image_path):
    with open(image_path, 'rb') as img_file:
        tags = exifread.process_file(img_file, details=False)
        return {tag: tags[tag].values for tag in tags}

def extract_png_textual_info(image):
    """ Extract textual information from PNG files """
    info = {}
    if hasattr(image, 'text'):
        for key, value in image.text.items():
            info[f"PNG Text: {key}"] = value
    return info

def extract_basic_info(image_path):
    with Image.open(image_path) as img:
        info = {
            "File Format": img.format,
            "Image Size": img.size,
            "Color Mode": img.mode,
            "Width (pixels)": img.width,
            "Height (pixels)": img.height
        }

        # Add PNG textual information if available
        if img.format == "PNG":
            info.update(extract_png_textual_info(img))

        # Add other properties from the 'info' dictionary
        for key, value in img.info.items():
            if key not in ["icc_profile", "dpi"]:
                info[f"Image Property: {key}"] = value

    return info

def save_to_excel(info, filename):
    df = pd.DataFrame([info])
    df.to_excel(filename, index=False)
    

current_dir = os.getcwd()

# Construct the path to the image folder
image_folder = 'image'
image_filename = 'image.png'  # Image file name
image_path = os.path.join(current_dir, image_folder, image_filename)

# Process the image
basic_info = extract_basic_info(image_path)

# Attempt to extract EXIF data if it's a JPEG or TIFF
if basic_info["File Format"] in ["JPEG", "TIFF"]:
    exif_data = extract_exif_data(image_path)
    basic_info.update(exif_data)

# Save the data to an Excel file
excel_filename = 'image_info.xlsx'
save_to_excel(basic_info, excel_filename)
print("Image exported to image_info.xlsx")
    

Image exported to image_info.xlsx


In [39]:
## image_analysis.py
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from PIL import Image, ImageFile
import numpy as np
import annotate_image

ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_and_prep_image(image_path, target_size=(224, 224)):
    """Load and prepare an image for analysis."""
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)
    img = np.array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

def model_predict(image_path, model):
    """Use a pre-trained model to predict the contents of an image."""
    img = load_and_prep_image(image_path)
    preds = model.predict(img)
    return decode_predictions(preds, top=5)[0]

def integrate_user_descriptions(predictions, user_descriptions):
    """Integrate user descriptions with model predictions."""
    user_words = set(user_descriptions.lower().split())
    integrated_data = {}

    for _, label, score in predictions:
        label_words = set(label.lower().replace('_', ' ').split())
        common_words = user_words.intersection(label_words)
        if common_words:
            integrated_data[label] = {
                "Model Confidence": f"{score:.2f}",
                "Related User Words": list(common_words)
            }

    return integrated_data

model = ResNet50(weights="imagenet")

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the image folder
image_folder = 'image'
image_filename = 'image.png'  # Replace this with your actual image file name
image_path = os.path.join(current_dir, image_folder, image_filename)

predictions = model_predict(image_path, model)

# User input for additional descriptions
user_descriptions = input("Enter your descriptions of the image: ")

# Integrate model predictions with user descriptions
integrated_info = integrate_user_descriptions(predictions, user_descriptions)
print("Integrated Data:", integrated_info)

# Placeholder for bounding box data
# This needs to be replaced with actual object detection data
boxes_example = [(100, 100, 200, 200), (300, 300, 400, 400)]
labels_example = [label for _, label, _ in predictions]

# Call the function from annotate_image.py to draw bounding boxes
annotate_image.draw_boxes(image_path, boxes_example, labels_example)


1/1 [==============================] - 1s 1s/step


Enter your descriptions of the image:  sheep


Integrated Data: {}


In [41]:
## annotate_image.py
from PIL import Image, ImageDraw, ImageFont, ImageFile

# Allow loading of truncated/corrupted images if possible
ImageFile.LOAD_TRUNCATED_IMAGES = True


def draw_boxes(image_path, boxes, labels):
    """Draw bounding boxes and labels on the image."""
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()

        for box, label in zip(boxes, labels):
            draw.rectangle(box, outline="red", width=2)
            draw.text((box[0], box[1]), label, fill="red", font=font)

        img.show()  # Or use img.save("annotated_image.jpg") to save the image
